In [72]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential

In [73]:
train_df = pd.read_csv('train_standard.csv')

In [74]:
train_df = train_df.drop('Unnamed: 0', axis=1)

In [75]:
train_df.head()

,id,cc_no,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt,cc_txn_amt
0,1,1,2,9.0,5,20000,1.000000,1750.000000,0.000000
1,1,98397,2,9.0,5,20000,1.000000,1750.000000,5000.000000
2,2,2,2,3.0,4,106000,4.346154,9111.538462,1034.615385
3,2,9740,2,3.0,4,106000,4.487805,9514.634146,797.560976
4,3,3,2,1.0,3,29000,3.333333,6652.380952,0.000000


In [76]:
train_id_df = pd.read_csv('train.csv')
train_id_df.head()

,id,income
0,1,20000
1,2,106000
2,3,29000
3,4,61000
4,5,18000


In [77]:
train_df = train_df.drop('cc_no', axis=1)
temp_group = train_df.groupby(['id', 'gender', 'ocp_cd', 'age', 'income']).mean()
temp_group

,,,,,kp_txn_count,kp_txn_amt,cc_txn_amt
id,gender,ocp_cd,age,income,,,
1,2,9.0,5,20000,1.000000,1750.000000,2500.000000
2,2,3.0,4,106000,4.416979,9313.086304,916.088180
3,2,1.0,3,29000,3.333333,6652.380952,0.000000
4,2,3.0,5,61000,12.191071,247372.142857,8785.714286
6,1,3.0,3,19000,3.333333,4466.666667,3833.333333
...,...,...,...,...,...,...,...
49995,1,4.0,6,132000,1.625000,5718.750000,793.750000
49996,2,3.0,4,48000,3.125000,7281.250000,246.875000
49997,1,9.0,5,37000,9.687692,27355.461538,59.615385


In [78]:
df = temp_group.reset_index()
df.head()

,id,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt,cc_txn_amt
0,1,2,9.0,5,20000,1.000000,1750.000000,2500.000000
1,2,2,3.0,4,106000,4.416979,9313.086304,916.088180
2,3,2,1.0,3,29000,3.333333,6652.380952,0.000000
3,4,2,3.0,5,61000,12.191071,247372.142857,8785.714286
4,6,1,3.0,3,19000,3.333333,4466.666667,3833.333333


In [79]:
len(df)

38046

In [80]:
len(df) * 0.8

30436.800000000003

In [81]:
len(df) * 0.2

7609.200000000001

In [82]:
indices = np.random.randint(len(df), size=len(df))
val = df.iloc[indices[:int(len(df)*0.2)]]
test = df.iloc[indices[int(len(df)*0.2):int(len(df)*0.2)*2]]
train = df.iloc[indices[int(len(df)*0.2)*2:]]

In [83]:
def split_x_y(df):
    X = pd.DataFrame()
    y = pd.DataFrame()
    cols = df.columns
    for col in cols:
        if col == 'income':
            y[col] = df[col]
        elif col == 'id':
            pass
        else:
            X[col] = df[col]
    X = X.to_numpy()
    y = y.to_numpy().reshape(-1)
    return X, y

In [95]:
x_val, y_val = split_x_y(val)
x_test, y_test = split_x_y(test)
x_train, y_train = split_x_y(train)
x_val.shape

(7609, 6)

In [89]:

model = Sequential([
    Dense(1024, activation='relu', input_shape=(6, )),
    Dropout(0.7),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')
])

model.summary()
model.compile(optimizer=Adam(learning_rate=0.1, decay=0.1), loss='mse', metrics=['mse'])

hist = model.fit(x_train, y_train, batch_size=1024, epochs=100,
                validation_data=(x_val, y_val), verbose=1, shuffle=True)


W1104 23:52:42.402085 4610329920 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W1104 23:52:42.602075 4610329920 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 1024)              7168      
_________________________________________________________________
dropout_12 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_25 (Dense)             (None, 128)               131200    
_________________________________________________________________
dropout_13 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 129       
Total params: 138,497
Trainable params: 138,497
Non-trainable params: 0
_________________________________________________________________
Train on 22828 samples, validate on 7609 samples
Epoch 1/100
22828/22828 [==============================] - 2s 81u

Epoch 41/100
22828/22828 [==============================] - 1s 33us/sample - loss: 4600615004.8992 - mse: 4600614912.0000 - val_loss: 4189707051.9059 - val_mse: 4189707264.0000
Epoch 42/100
22828/22828 [==============================] - 1s 35us/sample - loss: 4397565760.0561 - mse: 4397565440.0000 - val_loss: 4196353142.2600 - val_mse: 4196353280.0000
Epoch 43/100
22828/22828 [==============================] - 1s 35us/sample - loss: 4361440725.4754 - mse: 4361440256.0000 - val_loss: 4185406917.5261 - val_mse: 4185406976.0000
Epoch 44/100
22828/22828 [==============================] - 1s 35us/sample - loss: 4400462939.0152 - mse: 4400462848.0000 - val_loss: 4171491212.5998 - val_mse: 4171491072.0000
Epoch 45/100
22828/22828 [==============================] - 1s 35us/sample - loss: 4492169931.4722 - mse: 4492170240.0000 - val_loss: 4176729944.5183 - val_mse: 4176729856.0000
Epoch 46/100
22828/22828 [==============================] - 1s 36us/sample - loss: 4409916413.3086 - mse: 440991590

22828/22828 [==============================] - 1s 37us/sample - loss: 4528701861.8370 - mse: 4528701952.0000 - val_loss: 4129512248.3207 - val_mse: 4129512448.0000
Epoch 88/100
22828/22828 [==============================] - 1s 42us/sample - loss: 4478805466.3648 - mse: 4478805504.0000 - val_loss: 4128152821.2338 - val_mse: 4128152832.0000
Epoch 89/100
22828/22828 [==============================] - 1s 43us/sample - loss: 4536055843.3475 - mse: 4536055808.0000 - val_loss: 4136069375.9327 - val_mse: 4136069120.0000
Epoch 90/100
22828/22828 [==============================] - 1s 39us/sample - loss: 4420912337.5728 - mse: 4420912128.0000 - val_loss: 4135034015.3397 - val_mse: 4135033856.0000
Epoch 91/100
22828/22828 [==============================] - 1s 45us/sample - loss: 4455795688.8088 - mse: 4455795712.0000 - val_loss: 4130385975.6814 - val_mse: 4130386176.0000
Epoch 92/100
22828/22828 [==============================] - 1s 50us/sample - loss: 4334494080.1570 - mse: 4334494208.0000 - val_

In [91]:
pred = model.predict(x_train)

In [101]:
from sklearn import tree
clf = tree.DecisionTreeRegressor()
clf = clf.fit(x_train, y_train)
clf.predict(x_test)

array([48000., 36000., 14000., ..., 19000., 14000., 29000.])

In [117]:
y_test

array([ 48000, 127000,  17000, ...,  19000,  34000,  40000])

In [112]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=10, random_state=0, n_estimators=3000)
regr = clf.fit(x_train, y_train) 

In [116]:
pred = regr.predict(x_test)

In [119]:
eiei = (pred - y_test)
eiei

array([     0., -91000.,  -3000., ...,      0., -23000., -11000.])

In [123]:
eiei = np.array(eiei, dtype=np.int)
eiei

array([     0, -91000,  -3000, ...,      0, -23000, -11000])

In [124]:
eiei = np.where(eiei == 0, True, False)

In [126]:
eiei = list(eiei

AttributeError: 'numpy.ndarray' object has no attribute 'toList'